# Preamble

In [1]:
import xarray as xr
import numpy as np
import holoviews as hv
from pathlib import Path
import pandas as pd
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import numpy as np
from datetime import date
from matplotlib import pyplot as plt
# matplotlib widget
import dask.array as da
# from dask.distributed import Client, LocalCluster
# # import param
# from holoviews import streams
import csv
# Get working directory
import sys, os
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import marineHeatWaves as mhw
from marineHeatWaves import detect

## Import satellite data

In [2]:
# SET UO OF THE PARAMETERS THE CONTROL THE FILTER DATA

# First filter (buffer): cells close to the land
buffer = 0.8 # buffer size in kilometers

# Second filter (a): cells with strong difference with respect to the neighboring ones in a given time
temperature_threshold_a = 2 # Set your desired temperature difference threshold: decrease it to mask more

# Third filter (b): cells with strong difference with respect to the spatial average in a given time
temperature_threshold_b = 5 # Set your desired temperature difference threshold abs(T-T_spatial_average): decrease it to mask more

In [3]:
# linear interpolated datasets
fn_garda = 'interpolated/ID505-garda-LSWT-19920926_20221231-v2.1.0-DINEOF.interp.nc'

# set full paths
path_lswt_garda = Path(module_path).joinpath('../data/'+fn_garda)

# load lswt dataset as xarray.Dataset
ds_lswt_garda = xr.open_dataset(path_lswt_garda, chunks='auto')
ds_lswt_garda

<xarray.Dataset>
Dimensions:                         (time: 11054, lat: 52, lon: 43)
Coordinates:
  * time                            (time) datetime64[ns] 1992-09-26 ... 2022...
  * lat                             (lat) float64 45.45 45.45 ... 45.86 45.87
  * lon                             (lon) float64 10.51 10.52 ... 10.85 10.86
Data variables:
    lake_surface_water_temperature  (time, lat, lon) float32 dask.array<chunksize=(11054, 52, 43), meta=np.ndarray>
    distance_to_land                (lat, lon) float32 dask.array<chunksize=(52, 43), meta=np.ndarray>
    lakemask                        (lat, lon) bool dask.array<chunksize=(52, 43), meta=np.ndarray>
    qa_origin                       (lat, lon, time) int8 dask.array<chunksize=(52, 43, 11054), meta=np.ndarray>
    qa_cellgap                      (time, lat, lon) int16 dask.array<chunksize=(11054, 52, 43), meta=np.ndarray>
    qa_interpgap                    (time) int16 dask.array<chunksize=(11054,), meta=np.ndarray>
    qa_lakemask                     (lat, lon) int32 dask.array<chunksize=(52, 43), meta=np.ndarray>
    qa_dataavail                    (time) float64 dask.array<chunksize=(11054,), meta=np.ndarray>
Attributes: (12/17)
    lakename:               Garda
    lakeid:                 505
    lakecells:              462
    description:            Gap-filled lake surface water temperature derived...
    product_version:        v2.1.0
    gapfill_algorithm:      DINEOF
    ...                     ...
    setting_filt_buffer:    0.75
    setting_splits:         0
    dineof_ret_eofs:        39
    dineof_exp_error:       0.2251
    dineof_iters:           91
    dineof_convergence:     0.0009755

In [4]:
# access data variable using point
da_lswt_garda = ds_lswt_garda.lake_surface_water_temperature

# Slice based on data avilability

# Define time interval
start_date = '2007-01-01'
end_date = '2022-12-31'

# Slice the DataArray based on the time interval
da_lswt_garda = da_lswt_garda.sel(time=slice(start_date, end_date))

In [5]:
# Convert from Kelvin to Celcius
da_lswt_garda = da_lswt_garda - 273.15
da_lswt_garda.attrs['description'] = 'Lake Surface Water Temperature'
da_lswt_garda.attrs['unit'] = 'degrees Celsius (°C)'

In [6]:
# set-up a buffer to remove cells close to shore using the distance_to_land data variable

da_lswt_buffered_garda = da_lswt_garda.where(ds_lswt_garda['distance_to_land']>buffer)

In [7]:
# Function to calculate the second minimum absolute temperature difference
def calculate_second_min_absolute_difference(da, threshold, high_value=1000):
    # Create an array with surrounding points for each point
    surrounding_points = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1), (1, 0),  (1, 1)
    ]

    # Initialize the minimum and second minimum differences
    min_difference = np.inf
    second_min_difference = np.inf

    # Iterate through each surrounding point and calculate the absolute difference
    for offset_lat, offset_lon in surrounding_points:
        # Shift the data array by the offset
        shifted_da = da.shift(lat=offset_lat, lon=offset_lon)

        # Replace NaN values in the original and shifted arrays with a high value
        da_with_nan_replaced = da.where(~np.isnan(da), high_value)
        shifted_da_with_nan_replaced = shifted_da.where(~np.isnan(shifted_da), high_value)

        # Calculate the absolute difference between the original and shifted arrays
        temp_difference = np.abs(da_with_nan_replaced - shifted_da_with_nan_replaced)

        # Update the minimum and second minimum differences
        second_min_difference = np.minimum(np.maximum(min_difference, temp_difference), second_min_difference)
        min_difference = np.minimum(min_difference, temp_difference)

    # Create a mask based on the second minimum absolute difference criterion
    mask = (second_min_difference <= threshold) | (second_min_difference > high_value / 2)

    return mask


# Calculate the mask
mask_a = calculate_second_min_absolute_difference(da_lswt_buffered_garda, temperature_threshold_a, high_value=1000)
# Apply the mask to the original data array
da_lswt_buffered_masked_garda_a = da_lswt_buffered_garda.where(mask_a)

In [8]:
# Step 1: Calculate the spatial average temperature for each day
spatial_avg_temperature = da_lswt_buffered_masked_garda_a.mean(dim=['lat', 'lon'])
# Step 2: Calculate the absolute difference between spatial average and each cell's temperature
abs_difference_b = np.abs(da_lswt_buffered_masked_garda_a - spatial_avg_temperature)

# Step 4: Create a mask based on the absolute difference criterion
mask_b = abs_difference_b <= temperature_threshold_b
# Step 5: Apply the mask to the previously buffered and masked data array
da_lswt_buffered_masked_garda_b = da_lswt_buffered_masked_garda_a.where(mask_b)

# Satellite data Analisis

In [9]:
da_lswt = da_lswt_buffered_masked_garda_b
da_lswt

<xarray.DataArray 'lake_surface_water_temperature' (time: 5844, lat: 52, lon: 43)>
dask.array<where, shape=(5844, 52, 43), dtype=float32, chunksize=(5844, 52, 43), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2007-01-01 2007-01-02 ... 2022-12-31
  * lat      (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.85 45.86 45.87
  * lon      (lon) float64 10.51 10.52 10.53 10.54 ... 10.84 10.85 10.85 10.86
Attributes:
    description:  Lake Surface Water Temperature
    unit:         degrees Celsius (°C)

In [10]:
plot = da_lswt.hvplot(
    groupby='time',
    geo=True,
    cmap='jet', clim=(5, 30),
    xlabel='Longitude', ylabel='Latitude',
    clabel='LSWT (°C)',
    tiles='CartoLight',
    width=300,
    widget_type='scrubber', widget_location='bottom'
)
plot

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=5843, width=550)

In [11]:
# Count number of cells to analyze
# Count non-null values along the 'time' dimension
count_temp = da_lswt.count(dim='time')

# Find the cells that have temperature data at least once
cells_with_data = count_temp > 0

# Sum over the 'lat' and 'lon' dimensions
total_cells_with_data = cells_with_data.sum(dim=['lat', 'lon'])

print("Total number of cells containing temperature data over time:", int(total_cells_with_data))

Total number of cells containing temperature data over time: 452


In [12]:
# Extract the lat and lon indices where data are present
lat_lon_pairs = np.column_stack(np.where(cells_with_data))

# Import Modelled Series

In [13]:
# Function to format and import the simulated data
def import_and_format(lat_idx, lon_idx):
    file_path_cc = f'../air2water-master/Garda/output_4/2_PSO_RMS_{lat_idx}_{lon_idx}_cc_1d.out'
    file_path_cv = f'../air2water-master/Garda/output_4/3_PSO_RMS_{lat_idx}_{lon_idx}_cv_1d.out'
    
    # Import the data
    df_cc = pd.read_csv(file_path_cc, sep='\s+', header=None, usecols=[0, 1, 2, 5], names=['Year', 'Month', 'Day', 'Sim_Temp'])
    df_cv = pd.read_csv(file_path_cv, sep='\s+', header=None, usecols=[0, 1, 2, 5], names=['Year', 'Month', 'Day', 'Sim_Temp'])
    
    # Filter out rows where "Year" is equal to -999 (warming up year)
    df_cc = df_cc[df_cc['Year'] != -999]
    df_cv = df_cv[df_cv['Year'] != -999]
    
    # Reset index
    df_cc.reset_index(drop=True, inplace=True)
    df_cv.reset_index(drop=True, inplace=True)
    
    # Concat calibration and validation data
    df = pd.concat([df_cv, df_cc], ignore_index=True)
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    
    # Drop the individual year, month, and day columns
    df.drop(columns=['Year', 'Month', 'Day'], inplace=True)
    
    # Reorder
    df = df[['Date', 'Sim_Temp']]
    
    return df

# Dictionary to hold the formatted DataFrames
simulated_dataframes = {}

# Iterate through each pair of lat and lon indices
for lat_idx, lon_idx in lat_lon_pairs:
    df = import_and_format(lat_idx, lon_idx)
    simulated_dataframes[f"sim_df_{lat_idx}_{lon_idx}"] = df

# # Print the DataFrames
# for name, df in simulated_dataframes.items():
#     print(f"{name}:\n{df.head()}\n")


In [14]:
# Take the first DataFrame from the dictionary to initialize the DataArray
first_df = list(simulated_dataframes.values())[0]

# Initialize an empty DataArray with the new dimensions
sim_da_lswt = xr.DataArray(np.nan, coords=[first_df['Date'], da_lswt.lat, da_lswt.lon], dims=['time', 'lat', 'lon'])

# Iterate through the simulated_dataframes dictionary to assign the data
for name, df in simulated_dataframes.items():
    # Extract the latitude and longitude indices from the DataFrame name
    lat_idx, lon_idx = map(int, name.split('_')[2:])
    
    # Assign temperature data to the corresponding position in the DataArray
    sim_da_lswt.loc[dict(lat=da_lswt.lat.values[lat_idx], lon=da_lswt.lon.values[lon_idx])] = df['Sim_Temp'].values


# sim_da_lswt


In [15]:
plot = sim_da_lswt.hvplot(
    groupby='time',
    geo=True,
    cmap='jet', clim=(5, 30),
    xlabel='Longitude', ylabel='Latitude',
    clabel='LSWT (°C)',
    tiles='CartoLight',
    width=300,
    widget_type='scrubber', widget_location='bottom'
)
plot

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=11322, width=550)

In [16]:
# Function to calculate climatology using the marineHeatWaves detect function
def calculate_climatology(df):
    dates = df['Date'].apply(lambda x: x.toordinal()).values  # Convert dates to ordinal format
    temps = df['Sim_Temp'].values
    
    # Using the detect function to calculate climatology
    mhw_events, clim = mhw.detect(dates, temps)
    
    return clim

# Dictionary to hold the climatology results
climatologies = {}

# Iterate through the DataFrames and calculate the climatology
for name, df in simulated_dataframes.items():
    climatology = calculate_climatology(df)
    climatologies[name] = climatology

# # Print the climatologies
# for name, clim in climatologies.items():
#     print(f"{name} climatology:\n{clim}\n")


In [17]:
# Create the threshold dataArray
example_df = list(climatologies.values())[0]  # Get an example climatology DataFrame to get the time dimension
new_time_dim = len(example_df['thresh'])
thresh_values = np.full((new_time_dim, da_lswt.shape[1], da_lswt.shape[2]), np.nan)
seas_values = np.full((new_time_dim, da_lswt.shape[1], da_lswt.shape[2]), np.nan)


# Populate the array with threshold values from the climatologies
for name, clim in climatologies.items():
    lat_idx, lon_idx = map(int, name.split('_')[2:])
    thresh_values[:, lat_idx, lon_idx] = clim['thresh']

# Create the da_thresh DataArray
da_thresh = xr.DataArray(
    thresh_values,
    coords=[sim_da_lswt['time'], da_lswt.lat, da_lswt.lon],
    dims=['time', 'lat', 'lon']
)

# Populate the array with threshold values from the climatologies
for name, clim in climatologies.items():
    lat_idx, lon_idx = map(int, name.split('_')[2:])
    seas_values[:, lat_idx, lon_idx] = clim['seas']

# Create the da_thresh DataArray
da_clim = xr.DataArray(
    seas_values,
    coords=[sim_da_lswt['time'], da_lswt.lat, da_lswt.lon],
    dims=['time', 'lat', 'lon']
)

da_thresh

<xarray.DataArray (time: 11323, lat: 52, lon: 43)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ..., 9.42670635,
         9.28196406,        nan],
        [       nan,        nan,        nan, ..., 9.56951426,
         9.41870626, 9.17424923],
        [       nan,        nan,        nan, ..., 9.54059855,
         9.38348719,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ..., 9.5680441 ,
         9.41467181,        nan],
        [       nan,        nan,        nan, ..., 9.70353652,
         9.55241432, 9.30284568],
        [       nan,        nan,        nan, ..., 9.66782758,
         9.51279655,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ..., 9.49627668,
         9.34776245,        nan],
        [       nan,        nan,        nan, ..., 9.63536555,
         9.48510271, 9.23757213],
        [       nan,        nan,        nan, ..., 9.60321468,
         9.447863  ,        nan]]])
Coordinates:
  * time     (time) datetime64[ns] 1992-01-01 1992-01-02 ... 2022-12-31
  * lat      (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.85 45.86 45.87
  * lon      (lon) float64 10.51 10.52 10.53 10.54 ... 10.84 10.85 10.85 10.86

In [18]:
# Align the time coordinates
common_dates = np.intersect1d(da_lswt.time.values, da_thresh.time.values)

da_lswt_aligned = da_lswt.sel(time=common_dates)
da_thresh_aligned = da_thresh.sel(time=common_dates)
da_clim_aligned = da_clim.sel(time=common_dates)

# Ensure both arrays have the same time dimension
assert len(da_lswt_aligned.time) == len(da_thresh_aligned.time)

In [19]:
# Calculate intensities
da_intensity = da_lswt - da_clim_aligned
da_extrathresh = da_lswt - da_thresh_aligned
da_intensity

# Calculate categories
diff = da_thresh_aligned - da_clim_aligned

# Calculate category thresholds
category1_thresh = da_clim_aligned + 2 * diff
category2_thresh = da_clim_aligned + 3 * diff
category3_thresh = da_clim_aligned + 4 * diff

# Initialize the 'category' variable with NaNs
da_category = xr.full_like(da_lswt, np.nan, dtype=int)

# Assign category 0 where lswt is less than thresh, preserving NaN values
da_category = xr.where(da_lswt < da_thresh_aligned, 0, da_category)

da_category = xr.where(da_lswt > da_thresh_aligned, 1, da_category)
da_category = xr.where((da_lswt > category1_thresh) & (da_lswt <= category2_thresh), 2, da_category)
da_category = xr.where((da_lswt > category2_thresh) & (da_lswt <= category3_thresh), 3, da_category)
da_category = xr.where(da_lswt > category3_thresh, 4, da_category)

# Preserve NaN values from da_lswt
da_category = xr.where(da_lswt.isnull(), np.nan, da_category)

In [20]:
dataset_garda = xr.Dataset({
    'lswt': da_lswt,
    'clim': da_clim_aligned,
    'thresh': da_thresh_aligned,
    'intensity': da_intensity,
    'category': da_category,
    'extrathresh' : da_extrathresh
})
dataset_garda

<xarray.Dataset>
Dimensions:      (time: 5844, lat: 52, lon: 43)
Coordinates:
  * time         (time) datetime64[ns] 2007-01-01 2007-01-02 ... 2022-12-31
  * lat          (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.86 45.87
  * lon          (lon) float64 10.51 10.52 10.53 10.54 ... 10.85 10.85 10.86
Data variables:
    lswt         (time, lat, lon) float32 dask.array<chunksize=(5844, 52, 43), meta=np.ndarray>
    clim         (time, lat, lon) float64 nan nan nan nan ... 8.996 8.887 nan
    thresh       (time, lat, lon) float64 nan nan nan nan ... 9.603 9.448 nan
    intensity    (time, lat, lon) float64 dask.array<chunksize=(5844, 52, 43), meta=np.ndarray>
    category     (time, lat, lon) float64 dask.array<chunksize=(5844, 52, 43), meta=np.ndarray>
    extrathresh  (time, lat, lon) float64 dask.array<chunksize=(5844, 52, 43), meta=np.ndarray>

In [21]:
# Save the Dataset
dataset_garda.to_netcdf('../../output/DINEOF_lake_dataset_with_clim.nc')

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


In [22]:
custom_cmap = ['white', 'gold', 'darkorange', 'darkred', 'black']

plot2 = da_category.hvplot(
    groupby='time',
    geo=True,
    cmap= custom_cmap , clim=(-0.5, 4.5),
    xlabel='Longitude', ylabel='Latitude',
    clabel='Category (1:Moderate - 2:Strong - 3:Severe - 4:Extreme)',
    tiles='CartoLight',
    width=300,
    widget_type='scrubber', widget_location='bottom'
)
plot2

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=5843, width=550)

In [23]:
da_clim = dataset_garda.clim
da_clim

<xarray.DataArray 'clim' (time: 5844, lat: 52, lon: 43)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ..., 8.92219062,
         8.72312791,        nan],
        [       nan,        nan,        nan, ..., 9.0258728 ,
         8.87952409, 8.63886912],
        [       nan,        nan,        nan, ..., 8.93703551,
         8.826759  ,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ..., 9.05081764,
         8.84751952,        nan],
        [       nan,        nan,        nan, ..., 9.1526705 ,
         9.00440322, 8.75921707],
        [       nan,        nan,        nan, ..., 9.05628005,
         8.94931852,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ..., 8.98564993,
         8.78461365,        nan],
        [       nan,        nan,        nan, ..., 9.08847649,
         8.94125367, 8.69827576],
        [       nan,        nan,        nan, ..., 8.99591892,
         8.88736671,        nan]]])
Coordinates:
  * time     (time) datetime64[ns] 2007-01-01 2007-01-02 ... 2022-12-31
  * lat      (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.85 45.86 45.87
  * lon      (lon) float64 10.51 10.52 10.53 10.54 ... 10.84 10.85 10.85 10.86

In [24]:
plot = da_clim.hvplot(
    groupby='time',
    geo=True,
    cmap='jet', clim=(5, 20),
    xlabel='Longitude', ylabel='Latitude',
    clabel='LSWT (°C)',
    tiles='CartoLight',
    width=300,
    widget_type='scrubber', widget_location='bottom'
)
plot

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=5843, width=550)